In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import PySimpleGUI as sg
import re
import threading
import time

def scan():
    try:
        page = urllib.request.urlopen('http://jiofi.local.html')
        data=page.read()
        soup=BeautifulSoup(data,'html.parser')
        jiofi=soup.find_all('input')
        return jiofi
    except:
        time.sleep(3)
        scan()

def update():
    status={}
    for i in scan():
        try:
            ids = re.findall(r'["][\w\s]+["]',str(i))
            val_begin = re.search(r'value=',str(i)).end()
            value = str(i)[val_begin+1:-3]
            status[ids[0][1:-1]]=value
        except AttributeError:
            pass
    return status

def alert():
    status = update()
    level = status['batterylevel']
    if int(level[:-1]) < 30 and status['batterystatus']=='Discharging':
        sg.Popup("JioFi battery level under 30%")
    if int(level[:-1]) > 95 and status['batterystatus']=='Discharging':
        sg.Popup("Jiofi battery level exceded 95%")
        
def warn():
    while True:
        alert()
        time.sleep(10)

#t = threading.Thread(target=warn,args=())
#t.start()

warn()